In [ ]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns
sns.set()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [ ]:
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
client_id = "9cd3dd2ea2cf492ca28ab0247a79d781"
client_secret = "11c972ad002843e9be5ecc31f022dd6e"
grant_type = 'client_credentials'
body_params = {'grant_type' : grant_type}
url = 'https://accounts.spotify.com/api/token'

response = requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.content)

In [ ]:
# Using Spotipy library
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

sp.audio_features(["0UaMYEvWZi0ZqiDOoHU3YI"])

In [ ]:
def songs_in_playlist(df, pid):
    '''
    input: dataframe and playlist ID
    returns: list of all songs on playlist by track_uri 
    '''
    playlist = np.where(df.pid == pid)
    return list(df.iloc[playlist]["track_uri"])

def playlist_means(tracks):
    '''
    input: list of tracks on a playlist (can be generated from songs_in_playlist())
    returns: statistics for audio features of playlist  
    '''
    num = len(tracks)
    # We can only pass 100 tracks at a time
    if num <= 100:
        p = pd.DataFrame.from_records(sp.audio_features(tracks))
    else:
        start = 0
        end = 100
        p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
        for i in range(int(num / 100)):
            start = end
            diff = num - end
            if diff > 100:
                end += 100
            else:
                end += diff
            
            new_p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
            p = p.append(new_p) 
    return p

In [ ]:
songs = pd.read_pickle("pickles/songs_30k_dropped.pkl")
songs.head()

In [ ]:
songs2 = songs.copy()
songs2 = songs2.reset_index()
cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]

def scaler(cols, df_old, df_new):
    df_new = df_old.copy()
    x_scaler = MinMaxScaler().fit(df_old["tempo"].values.reshape(-1,1))
    x_scaled = x_scaler.transform(df_old["tempo"].values.reshape(-1,1))
    df_new["tempo"] = x_scaled

#     x_scaler.transform(df_old[col].values.reshape(-1,1))
    
#     for col in cols: 
#         x_scaler = MinMaxScaler().fit(df_old[col].values.reshape(-1,1))
#         x_scaled = MinMaxScaler().fit_transform(df_old[col].values.reshape(-1,1))
#         df_new[col] = x_scaled
        
#         x_scaler.transform(df_old[col].values.reshape(-1,1))
    return(df_new, x_scaler)


        
songs2, x_scaler = scaler(cols, songs, songs2)
songs2.head()
songs2['index']

In [ ]:
song1 = songs2.iloc[np.where(songs.id == '1MYYt7h6amcrauCOoso3Gx')]
song1

In [ ]:
# def distance(idkey, songs):
#     song = pd.DataFrame(sp.audio_features(idkey))
#     song_scaled = song.copy()
# #     song = scaler(cols, song, song_scaled)
#     for col in ["tempo"]:
#         song_scaled[col] = x_scaler.transform(song.loc[:, col].values.reshape(-1,1))
#         print(col, song_scaled.loc[:, col])
        
#     song = song_scaled
#     dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
#                  + (np.subtract(songs.danceability, song.danceability))**2
#                  + (np.subtract(songs.energy, song.energy))**2
#                  + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
#                  + (np.subtract(songs.liveness, song.liveness))**2
#                  + (np.subtract(songs.speechiness, song.speechiness))**2
#                  + (np.subtract(songs.tempo, song.tempo))**2)
        
#     return dist


def playlist_generator(input_song_uri, n, songDF):
    def distance(idkey, songs):
        song = pd.DataFrame(sp.audio_features(idkey))
        song.loc[:,"tempo"] = x_scaler.transform(song.loc[:,"tempo"].values.reshape(-1,1))
        dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
                     + (np.subtract(songs.danceability, song.danceability))**2
                     + (np.subtract(songs.energy, song.energy))**2
                     + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
                     + (np.subtract(songs.liveness, song.liveness))**2
                     + (np.subtract(songs.speechiness, song.speechiness))**2
                     + (np.subtract(songs.tempo, song.tempo))**2)

        return dist
    
    topn_dist = distance(input_song_uri, songDF).sort_values()[0:n]
    print(topn_dist.index)
    topn = songs2.loc[topn_dist.index]
    return topn

def playlist_printer(topn_ids):
    for ref in topn_ids['id']:
        track = sp.track(ref)
        print(track['name'])
        print('by')
        for artist in track['artists']:
            print(artist['name'])
        print('-----------------------------------')
        

topn_ids = playlist_generator('5cYA45RVGI6F4f06gtWjsd', 25, songs2)
playlist_printer(topn_ids)

In [ ]:
top10_dist = distance('5IVuqXILoxVWvWEPm82Jxr', songs2).sort_values()[0:10]

In [ ]:
top10_dist.index